# ExaOne

In [1]:
import time
import textworld
import numpy as np
import re

In [2]:
import torch
import accelerate
torch.set_default_device('cuda')
torch.cuda.device("cuda")
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision('high')

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-1.7B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/622M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [8]:
print(model.hf_device_map)

{'model.embed_tokens': 0, 'lm_head': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 0, 'model.layers.7': 0, 'model.layers.8': 0, 'model.layers.9': 0, 'model.layers.10': 0, 'model.layers.11': 0, 'model.layers.12': 0, 'model.layers.13': 0, 'model.layers.14': 0, 'model.layers.15': 0, 'model.layers.16': 0, 'model.layers.17': 0, 'model.layers.18': 0, 'model.layers.19': 0, 'model.layers.20': 0, 'model.layers.21': 0, 'model.layers.22': 0, 'model.layers.23': 0, 'model.layers.24': 'cpu', 'model.layers.25': 'cpu', 'model.layers.26': 'cpu', 'model.layers.27': 'cpu', 'model.norm': 'cpu', 'model.rotary_emb': 'cpu'}


In [5]:
think_command_id = 151668

In [17]:
# prepare the model input
prompt = "Explain what the optimal strategy for winning Space Invaders is in one sentence. /no_think"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
start = time.time()
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32000
)
end = time.time()
print(f"(inference took {(end - start):.3f} seconds)")

output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # index finding </think>
    index = len(output_ids) - output_ids[::-1].index(think_command_id)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

(inference took 6.653 seconds)
thinking content: <think>

</think>
content: The optimal strategy for winning Space Invaders is to shoot the invaders when they are in the middle of the screen, move left or right to avoid them, and shoot them when they are close to the edges to maximize damage.


In [14]:
print(tokenizer.chat_template)

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0].role == 'system' %}
        {{- messages[0].content + '\n\n' }}
    {%- endif %}
    {{- "# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0].role == 'system' %}
        {{- '<|im_start|>system\n' + messages[0].content + '<|im_end|>\n' }}
    {%- endif %}
{%- endif %}
{%- set ns = namespace(multi_step_tool=true, last_query_index=messages|length - 1) %}
{%- for message in messages[::-1] %}
    {%- set index = (messages|length - 

## Context size and shifting window

In [ ]:
token_system = "[|system|]"
token_endofturn = "[|endofturn|]"
token_user = "[|user|]"
token_assistant = "[|assistant|]"

In [ ]:
!echo "Downloading zork1.z5 ..."
!wget -q -N https://archive.org/download/Zork1Release88Z-machineFile/zork1.z5
!echo "Done."

In [ ]:
# Let the environment know what information we want as part of the game state.
infos = textworld.EnvInfos(
    feedback=True,    # Response from the game after typing a text command.
    description=True, # Text describing the room the player is currently in.
    inventory=True    # Text describing the player's inventory.
)

In [ ]:
env = textworld.start('./zork1.z5', request_infos=infos)

In [ ]:
system_prompt = "You are an assistant playing a textual game. You analyze the information given carefully and reply exclusively in the form \"verb noun\", e.g. \"open box\" or \"take key\"."
context = token_system + system_prompt + token_endofturn

try:
    done = False
    env.reset()
    while not done:
        game_status = env.render(mode="text")
        print(game_status)
        context += token_user + game_status + token_assistant
        
        start = time.time()    
        input_ids = tokenizer.encode(
            context,
            return_tensors="pt")
        
        output = model.generate(
            input_ids.to("cuda"),
            eos_token_id=tokenizer.eos_token_id,
            max_new_tokens=10,
            do_sample=False)

        response = tokenizer.decode(output[0, input_ids.shape[1]:])
        context += response
        print(response.split("[")[0])
        reply = re.sub('\W+',' ', response.split("[")[0]) # # remove token endofturn and potential unwanted characters, like quotes
        
        end = time.time()
        print(f"Inference took {(end - start):.3f} seconds")
        
        command = reply if len(reply.split()) <= 4 else "look around"
        game_state, reward, done = env.step(command)

    env.render()  # Final message.
except KeyboardInterrupt:
    pass  # Quit the game.

print("Played {} steps, scoring {} points.".format(game_state.moves, game_state.score))

Next: use the notebook `Playing TextWorld generated games with OpenAI Gym.ipynb`